<a href="https://colab.research.google.com/github/TXH2020/MainRepo/blob/main/Projects/Python%20SQL/Python_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Python with MYSQL:

In [ ]:
!apt-get update
!apt install mysql-server

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,083 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,371 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-updates/restr

In [ ]:
!sudo service mysql start

 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 32.6 MB/s eta 0:00:00


###To configure the mysql server
###Run the below cell
###After login, type: use mysql;
###Then execute the below command:
###ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'jtp12345';
###Then type exit

In [ ]:
!mysql -u root

Welcome to the MySQL monitor.  Commands end with ; or \g.
Your MySQL connection id is 10
Server version: 8.0.33-0ubuntu0.20.04.2 (Ubuntu)

Copyright (c) 2000, 2023, Oracle and/or its affiliates.

Oracle is a registered trademark of Oracle Corporation and/or its
affiliates. Other names may be trademarks of their respective
owners.

Type 'help;' or '\h' for help. Type '\c' to clear the current input statement.

mysql> ALTER USER 'root'@'localhost' IDENTIFIED WITH mysql_native_password BY 'j tp12345';
Query OK, 0 rows affected (0.01 sec)

mysql> exit
Bye


In [ ]:
#Python MySQL
import requests
response=requests.get("https://raw.githubusercontent.com/TXH2020/MainRepo/main/Projects/Python%20SQL/example.sql")
with open("mysql.sql",'wb') as file:
    file.write(response.content)

import mysql.connector as my
try:
 cnx=my.connect(user="root",host="localhost",password="jtp12345")
except my.Error as e:
 print(e)
else:
    file=open('mysql.sql','r',encoding='latin-1')
    s=file.read()
    file.close()
    l=s.split(';')
    cr=cnx.cursor()
    for i in l:
        try:
            cr.execute(i,multi=True)
        except my.Error as e:
            print(e.msg)
    cnx.commit()
    cnx.close()

try:
     cnx = my.connect(user="root",host="localhost",password="jtp12345")
except my.Error as e:
 print(e)
else:
 cr=cnx.cursor()
 print("******************")
 cr.execute("use python_mysql")
 cr.execute("Select *from books")    #Manipulate database
 k=0
 for i in cr:
     k=k+1
     print(i)
 print("No. of records=", k)
 print("******************")

 cr.execute("show columns from books")
 for i in cr:
     print(i[0])                         #Showing column names
 print("******************")

 add_book = ("INSERT INTO books"
                 "(id, title, isbn)"
                 "VALUES (%s, %s, %s)")
 book = (86,'Hello World',"1234567898784")
 cr.execute(add_book,book)            #Inserting record
 cr.execute("select count(id) from books")
 print("Number of records after inserting is=",cr.fetchall())
 print("*******************")
 cr.execute("delete from books where id=86")
 cr.execute("select count(id) from books")
 print("Number of records after deleting row with id=86 is=", cr.fetchall())
 cnx.close()
#Like this we can use any Mysql command in Python using mysql connector.

******************
(1, 'Bel and the Dragon ', '123828863494')
(2, 'Daughters of Men ', '1234404543724')
(3, 'The Giant on the Hill ', '1236400967773')
(4, 'Marsh Lights ', '1233673027750')
(5, 'Mr. Wodehouse and the Wild Girl ', '1232423190947')
(6, 'The Fairy Castle ', '1237654836443')
(7, 'The Girl Who Walked a Long Way ', '1230211946720')
(8, 'The Runaway ', '1238155430735')
(9, 'The Shrubbery ', '1237366725549')
(10, 'Tom Underground a play ', '1239633328787')
(11, 'Anemones of the British Coast ', '1233540471995')
(12, 'Ask to Embla poem-cycle ', '1237417184084')
(13, 'Cassandra verse drama ', '1235260611012')
(14, 'Chidiock Tichbourne ', '1230468662299')
(15, 'The City of Is ', '1233136349197')
(16, 'Cromwell verse drama ', '1239653041219')
(17, 'Debatable Land Between This World and the Next ', '1235927658929')
(18, 'The Fairy Melusina epic poem ', '1232341278470')
(19, 'The Garden of Proserpina ', '1234685512892')
(20, 'Gods Men and Heroes ', '1233369260356')
(21, 'The Great Co

In [ ]:
!sudo service mysql stop

 * Stopping MySQL database server mysqld
   ...done.


##Python with NoSQL(MongoDB)

In [ ]:
!sudo apt-get install mongodb

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpcap0.8 libyaml-cpp0.6 mongo-tools mongodb-clients mongodb-server
  mongodb-server-core
The following NEW packages will be installed:
  libpcap0.8 libyaml-cpp0.6 mongo-tools mongodb mongodb-clients mongodb-server
  mongodb-server-core
0 upgraded, 7 newly installed, 0 to remove and 15 not upgraded.
Need to get 55.8 MB of archives.
After this operation, 226 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libpcap0.8 amd64 1.9.1-3 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libyaml-cpp0.6 amd64 0.6.2-4ubuntu1 [124 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 mongo-tools amd64 3.6.3-0ubuntu1 [12.3 MB]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 mongodb-clients amd64 1:3.6.9+really3.6.8+90~g8e540c0b6d-0ubuntu5.3 [21.6 MB]
Get:5 

In [ ]:
!sudo service mongodb start

 * Starting database mongodb
   ...done.


In [ ]:
!pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 22.7 MB/s eta 0:00:00


In [ ]:
#Python with mongodb nosql
import pandas as pd
import pymongo as pm
import numpy as np
df = pd.read_csv (r'https://raw.githubusercontent.com/TXH2020/MainRepo/main/Projects/Python%20SQL/listings.csv')
client = pm.MongoClient('mongodb://localhost:27017/')
mydb=client['somedatabase']
if 'somecollection' in mydb.list_collection_names():
    mydb['somecollection'].drop()
mycol=mydb['somecollection']
for i in range(df.shape[0]):
    r=dict(df.iloc[i])
    try:
        mycol.insert_one(r)
    except:
        # Python 2.7.10 on Windows and Pymongo are not forgiving
        # If you have foreign data types you have to convert them
        n = {}
        for k, v in r.items():
            if isinstance(v, np.int64):
                v = int(v)
            elif isinstance(v,np.float64):
                v=float(v)
            n[k]=v
        mycol.insert_one(n)
print("No of documents=",mycol.count_documents({}))
print("Document with id=2818")
print('---------------------------')
doc=mycol.find({'id':2818})
for i in doc:
    for j in i.items():
        print(j)
client.close()

No of documents= 6893
Document with id=2818
---------------------------
('_id', ObjectId('64b0116bf12a7231743ded8c'))
('id', 2818)
('listing_url', 'https://www.airbnb.com/rooms/2818')
('scrape_id', 20220907151523)
('last_scraped', '2022-09-07')
('source', 'city scrape')
('name', 'Quiet Garden View Room & Super Fast WiFi')
('description', 'Quiet Garden View Room & Super Fast WiFi<br /><br /><b>The space</b><br />I\'m renting a bedroom (room overlooking the garden) in my apartment in Amsterdam, <br /><br />The room is located to the east of the city centre in a quiet, typical Amsterdam neighbourhood the "Indische Buurt". Amsterdam’s historic centre is less than 15 minutes away by bike or tram.<br /><br /><br />The features of the room are:<br /><br />- Twin beds (80 x 200 cm, down quilts and pillows) <br />- 2 pure cotton towels for each guest <br />- reading lamps<br />- bedside table<br />- wardrobe<br />- table with chairs<br />- tea and coffee making facilities<br />- mini bar<br />-

In [ ]:
!sudo service mongodb stop

 * Stopping database mongodb
   ...done.


##Apache Cassandra:

In [ ]:
!wget https://dlcdn.apache.org/cassandra/4.0.10/apache-cassandra-4.0.10-bin.tar.gz

--2023-07-13 15:01:30--  https://dlcdn.apache.org/cassandra/4.0.10/apache-cassandra-4.0.10-bin.tar.gz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48905667 (47M) [application/x-gzip]
Saving to: ‘apache-cassandra-4.0.10-bin.tar.gz’

apache-cassandra-4. 100%[===================>]  46.64M   170MB/s    in 0.3s    

2023-07-13 15:01:30 (170 MB/s) - ‘apache-cassandra-4.0.10-bin.tar.gz’ saved [48905667/48905667]



In [ ]:
!tar xzvf apache-cassandra-4.0.10-bin.tar.gz

apache-cassandra-4.0.10/bin/
apache-cassandra-4.0.10/conf/
apache-cassandra-4.0.10/conf/triggers/
apache-cassandra-4.0.10/doc/
apache-cassandra-4.0.10/doc/cql3/
apache-cassandra-4.0.10/lib/
apache-cassandra-4.0.10/lib/sigar-bin/
apache-cassandra-4.0.10/pylib/
apache-cassandra-4.0.10/pylib/cqlshlib/
apache-cassandra-4.0.10/pylib/cqlshlib/test/
apache-cassandra-4.0.10/pylib/cqlshlib/test/config/
apache-cassandra-4.0.10/tools/
apache-cassandra-4.0.10/tools/bin/
apache-cassandra-4.0.10/tools/lib/
apache-cassandra-4.0.10/CASSANDRA-14092.txt
apache-cassandra-4.0.10/CHANGES.txt
apache-cassandra-4.0.10/LICENSE.txt
apache-cassandra-4.0.10/NEWS.txt
apache-cassandra-4.0.10/NOTICE.txt
apache-cassandra-4.0.10/conf/README.txt
apache-cassandra-4.0.10/conf/cassandra-env.sh
apache-cassandra-4.0.10/conf/cassandra-jaas.config
apache-cassandra-4.0.10/conf/cassandra-rackdc.properties
apache-cassandra-4.0.10/conf/cassandra-topology.properties
apache-cassandra-4.0.10/conf/cassandra.yaml
apache-cassandra-4.0.

In [ ]:
!apache-cassandra-4.0.10/bin/cassandra -R

OpenJDK 64-Bit Server VM warning: Option UseConcMarkSweepGC was deprecated in version 9.0 and will likely be removed in a future release.
CompileCommand: dontinline org/apache/cassandra/db/Columns$Serializer.deserializeLargeSubset(Lorg/apache/cassandra/io/util/DataInputPlus;Lorg/apache/cassandra/db/Columns;I)Lorg/apache/cassandra/db/Columns;
CompileCommand: dontinline org/apache/cassandra/db/Columns$Serializer.serializeLargeSubset(Ljava/util/Collection;ILorg/apache/cassandra/db/Columns;ILorg/apache/cassandra/io/util/DataOutputPlus;)V
CompileCommand: dontinline org/apache/cassandra/db/Columns$Serializer.serializeLargeSubsetSize(Ljava/util/Collection;ILorg/apache/cassandra/db/Columns;I)I
CompileCommand: dontinline org/apache/cassandra/db/commitlog/AbstractCommitLogSegmentManager.advanceAllocatingFrom(Lorg/apache/cassandra/db/commitlog/CommitLogSegment;)V
CompileCommand: dontinline org/apache/cassandra/db/transform/BaseIterator.tryGetMoreContents()Z
CompileCommand: dontinline org/apache/c

In [ ]:
!pip3 install cassandra-driver

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 37.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/TXH2020/MainRepo/main/Projects/Python%20SQL/data.csv",encoding="latin1",header=None)
df.columns=['id','title','isbn']
df['isbn']=df['isbn'].apply(lambda x:str(x))
df['title']=df['title'].apply(lambda x:x.replace("'",' '))

In [ ]:
from cassandra.cluster import Cluster
clstr=Cluster()
session=clstr.connect()
session.execute("""create keyspace mykeyspace with replication={
   'class': 'SimpleStrategy', 'replication_factor' : 1
};""")


In [ ]:
session=clstr.connect('mykeyspace')
qry= """
create table data (
   id int,
   name text,
   isbn text,
   primary key(id))"""
session.execute(qry)

In [ ]:
q="""insert into data (id,name,isbn) values """
for i in range(df.shape[0]):
  try:
    session.execute(q+str(tuple(df.iloc[i])))
  except:
    pass

In [ ]:
q1="select count(*) from data"
row=session.execute(q1)
print("count = ",str(row.current_rows)[11:13])

count =  83


In [ ]:
q2="select * from data;"
rows=session.execute(q2)
l=[]
for row in rows:
  l.append(list(row))

In [ ]:
l.sort(key=lambda x:x[0])

In [ ]:
print("Result of select command")
for i in l:
  print(i[0],i[1],i[2])

Result of select command
1 123828863494 Bel and the Dragon 
2 1234404543724 Daughters of Men 
3 1236400967773 The Giant on the Hill 
4 1233673027750 Marsh Lights 
5 1232423190947 Mr. Wodehouse and the Wild Girl 
6 1237654836443 The Fairy Castle 
7 1230211946720 The Girl Who Walked a Long Way 
8 1238155430735 The Runaway 
9 1237366725549 The Shrubbery 
10 1239633328787 Tom Underground a play 
11 1233540471995 Anemones of the British Coast 
12 1237417184084 Ask to Embla poem-cycle 
13 1235260611012 Cassandra verse drama 
14 1230468662299 Chidiock Tichbourne 
15 1233136349197 The City of Is 
16 1239653041219 Cromwell verse drama 
17 1235927658929 Debatable Land Between This World and the Next 
18 1232341278470 The Fairy Melusina epic poem 
19 1234685512892 The Garden of Proserpina 
20 1233369260356 Gods Men and Heroes 
21 1237871538785 The Great Collector 
22 1234003421055 The Grecian Way of Love 
23 1233804025236 The Incarcerated Sorceress 
24 1231588537286 Last Tales 
25 1239338429682 L

In [ ]:
#Kill the cassandra server
import psutil
# Iterate over all running process
for proc in psutil.process_iter():
    try:
        # Get process name & pid from process object.
        processName = proc.name()
        if(processName=='java'):
          proc.kill()
          break
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

##Python with Cloud SQL(Azure SQL DB)

In [ ]:
!pip install pyodbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


     |████████████████████████████████| 472 kB 4.3 MB/s 


In [ ]:
#Python with Azure Cloud SQL. However one must create an SQL server in Azure which isn't free.
#Make sure that the database is created with sample data(AdventureWorksLT)

import pyodbc
server = 'tezserver.database.windows.net' #Enter your respective server name
database = 'somedatabase'                 #Enter your respective database name
username = 'Moto12'                       #Enter your respective usernname and password for SQL server
password = 'Hello123*'
driver= '{SQL Server}'

try:
  with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';PORT=1433;DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    with conn.cursor() as cursor:
        cursor.execute("select * from [SalesLT].[Product]")
        row = cursor.fetchone()
        while row:
            print (str(row[0]) + " " + str(row[1]))
            row = cursor.fetchone()
except:
  print("Sorry the database does not exist")

Sorry the database does not exist
